In [1]:
import numpy as np
import pandas as pd
from openmm.unit import bar, mole, litre, kelvin, kilojoule_per_mole, nanometer, angstrom, kilocalorie_per_mole, kilogram, molar, atmosphere, nanosecond, picosecond, femtoseconds
from openmm.unit import Quantity, Unit
from openmm.unit import AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
import math

In [2]:
#Volume in L of Box
w=4.8*nanometer
l=4.8*nanometer
h=4.8*nanometer
vol=(w*l*h)
volL=vol.in_units_of(litre)
print(f'Volume is {volL} L')

#Particles of water in box
waters=(55.55*molar)*volL*AVOGADRO_CONSTANT_NA
print(f'Number of water molecules in box: {math.ceil(waters)}')

Volume is 1.10592e-22 L L
Number of water molecules in box: 3700


In [3]:
#Molarity calculation from molality, particle number
def particles(m,dens,mm_solute):
    grams_solute=m*mm_solute
    grams_soln=1000+grams_solute
    liters_soln=(grams_soln/dens)/1000
    molarity_final=m/liters_soln
    moles=molarity_final*vol
    molecules=moles*6.022E23
    mole_fctn=55.55/(molarity_final+55.55)
    return print('Molality is '+'%.2f'%m+'m'+' and Molarity is '+'%.2f'%molarity_final+'M'), print('Number of particles of solute: '+'%.0f'%molecules), print('Mole Fraction: '+'%.3f'%mole_fctn)

In [4]:
def solute_particles(num_water_molecules, molality_molkg):
    """
    Calculate the number of solute molecules required to achieve a target molality.

    Parameters:
    - num_water_molecules (int): Total number of water molecules in the system.
    - molality_molkg (float): Target molality in mol/kg.

    Returns:
    - int: Number of solute molecules needed.
    """
    AVOGADRO = 6.02214076e23  # molecules/mol
    WATER_MOLAR_MASS = 18.01528  # g/mol
    KG_PER_GRAM = 1e-3  # Conversion factor

    # Calculate the mass of water in grams
    mass_water_grams = (num_water_molecules / AVOGADRO) * WATER_MOLAR_MASS

    # Convert mass of water to kilograms
    mass_water_kg = mass_water_grams * KG_PER_GRAM

    # Use molality definition: mol/kg = moles_solute / mass_water_kg
    moles_solute = molality_molkg * mass_water_kg

    # Convert moles of solute to number of molecules
    num_solute_molecules = int(round(moles_solute * AVOGADRO))

    return num_solute_molecules


In [5]:
molalities=[0.1,0.5,1.0,1.4,2.0,2.5,3.0,3.5,4.0]

In [6]:
solute_particle_counts = []
for i in molalities:
    particles_needed = solute_particles(num_water_molecules=math.ceil(waters), molality_molkg=i)
    solute_particle_counts.append(particles_needed)
    print(f"Number of solute molecules needed for {i} molal: {particles_needed}")

Number of solute molecules needed for 0.1 molal: 7
Number of solute molecules needed for 0.5 molal: 33
Number of solute molecules needed for 1.0 molal: 67
Number of solute molecules needed for 1.4 molal: 93
Number of solute molecules needed for 2.0 molal: 133
Number of solute molecules needed for 2.5 molal: 167
Number of solute molecules needed for 3.0 molal: 200
Number of solute molecules needed for 3.5 molal: 233
Number of solute molecules needed for 4.0 molal: 267


In [7]:
def generate_packmol_inputs(salt_tag, ion1, ion2, molality_tag, water_blocks, ion_count, replicate_range, output_dir="."):
    """
    Generates multiple PACKMOL input strings with varying replicate indices.

    Parameters:
    - salt_tag (str): Short salt name, e.g., 'csbr'
    - molality_tag (str): Molality string, e.g., '01m'
    - water_blocks (list of int): List of water molecule counts per block
    - ion_count (int): Number of cation and anion molecules
    - replicate_range (range): e.g., range(0, 21)
    - output_dir (str): Directory to save generated inputs (optional)
    """
    for r in replicate_range:
        lines = [
            f"#",
            f"# {molality_tag} {salt_tag.upper()} and water",
            f"#",
            f"seed -1",
            f"tolerance 2.0",
            ""
        ]

        # Add water blocks (layered along Z)
        for i, water_num in enumerate(water_blocks):
            zmin = i * 48
            zmax = (i + 1) * 48
            lines.extend([
                f"structure ../structures/water.pdb",
                f"  number {water_num}",
                f"  inside box 0. 0. {zmin} 48. 48. {zmax}.",
                f"end structure",
                ""
            ])

        # Add cation and anion in middle region
        zmid_min = 48
        zmid_max = 96
        for ion in [ion1, ion2]:
            lines.extend([
                f"structure ../structures/{ion}.pdb",
                f"  number {ion_count}",
                f"  inside box 0. 0. {zmid_min} 48. 48. {zmid_max}.",
                f"end structure",
                ""
            ])

        # Add output
        output_name = f"{salt_tag}_{molality_tag}_r{r}.pdb"
        lines.append(f"output {output_name}")

        # Save or return the input
        input_text = "\n".join(lines)
        file_path = f"{output_dir}/{salt_tag}_{molality_tag}_r{r}.inp"
        with open(file_path, "w") as f:
            f.write(input_text)

        print(f"Generated: {file_path}")


In [ ]:
for mol,i in zip(molalities,solute_particle_counts):
    if mol % 1 == 0:
        mi1 = f"{mol:.1f}"
        mi = str(int(mol))  # Whole number: strip decimal
    else:
        mi = f"{mol:.1f}".replace('.', '')  # Float: remove the dot
        mi1 = f"{mol:.1f}"  # used in salt_dict key lookup
    generate_packmol_inputs(
    salt_tag="mgcl2",
    ion1="mg",
    ion2="cl",
    molality_tag=f'{str(mi)}m',
    water_blocks=[3700, 3700, 3700],
    ion_count=i,
    replicate_range=range(0, 21),
    output_dir="./packmol_inputs",
    double_ion="cl"  # Doubles Cl⁻ to 14
)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (730940644.py, line 14)

In [11]:
from concurrent.futures import ThreadPoolExecutor
import subprocess
import glob
import os
import re

def run_packmol(inp_file):
    # Read .inp file and check for output line
    with open(inp_file, "r") as f:
        lines = f.readlines()

    # Try to find the output file line
    output_file = None
    for line in lines:
        match = re.match(r"\s*output\s+(\S+)", line, re.IGNORECASE)
        if match:
            output_file = match.group(1)
            break

    # If output file already exists, skip running
    if output_file and os.path.exists(output_file):
        print(f"⏩ Skipping {inp_file}, output already exists: {output_file}")
        return

    # Run packmol
    with open(inp_file, "r") as f:
        print(f"▶️ Running: {inp_file}")
        result = subprocess.run(["packmol"], stdin=f, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Error in {inp_file}:\n{result.stderr}")
        else:
            print(f"✅ Finished: {inp_file}")

inp_files = glob.glob("packmol_inputs/*.inp")

with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(run_packmol, inp_files)


⏩ Skipping packmol_inputs/nano3_25m_r12.inp, output already exists: nano3_25m_r12.pdb
⏩ Skipping packmol_inputs/mgcl_01m_r11.inp, output already exists: mgcl_01m_r11.pdb
⏩ Skipping packmol_inputs/nano3_05m_r1.inp, output already exists: nano3_05m_r1.pdb
⏩ Skipping packmol_inputs/mgcl_05m_r0.inp, output already exists: mgcl_05m_r0.pdb
⏩ Skipping packmol_inputs/nano3_1m_r19.inp, output already exists: nano3_1m_r19.pdb
⏩ Skipping packmol_inputs/nh4no3_14m_r16.inp, output already exists: nh4no3_14m_r16.pdb
⏩ Skipping packmol_inputs/nano3_4m_r0.inp, output already exists: nano3_4m_r0.pdb
⏩ Skipping packmol_inputs/nano3_3m_r15.inp, output already exists: nano3_3m_r15.pdb
⏩ Skipping packmol_inputs/mgcl_1m_r9.inp, output already exists: mgcl_1m_r9.pdb
⏩ Skipping packmol_inputs/mgcl_25m_r7.inp, output already exists: mgcl_25m_r7.pdb
⏩ Skipping packmol_inputs/nacl_4m_r4.inp, output already exists: nacl_4m_r4.pdb
⏩ Skipping packmol_inputs/nh4no3_14m_r18.inp, output already exists: nh4no3_14m_r18.

In [ ]:
# from concurrent.futures import ThreadPoolExecutor
# import subprocess
# import glob

# def run_packmol(inp_file):
#     with open(inp_file, "r") as f:
#         print(f"Running: {inp_file}")
#         result = subprocess.run(["packmol"], stdin=f, capture_output=True, text=True)
#         if result.returncode != 0:
#             print(f"❌ Error in {inp_file}:\n{result.stderr}")
#         else:
#             print(f"✅ Finished: {inp_file}")

# inp_files = glob.glob("packmol_inputs/*.inp")

# with ThreadPoolExecutor(max_workers=4) as executor:
#     executor.map(run_packmol, inp_files)


In [12]:
ions_df = pd.read_csv('ion_densities.csv')

In [13]:
ions_df

,Salt,Molar Mass (g/mol),Molality (kg/mol),Density (g/mL),Exp Osmotic Coefficient
0,CsBr,212.810,0.1,1.01356,0.917
1,CsBr,212.810,0.2,1.02986,0.896
2,CsBr,212.810,0.3,1.04598,0.882
3,CsBr,212.810,0.4,1.06192,0.873
4,CsBr,212.810,0.5,1.07769,0.865
...,...,...,...,...,...
534,NH42HP,115.025,1.5,1.08100,0.658
535,NH42HP,115.025,2.0,1.10600,0.630
536,NH42HP,115.025,2.5,1.12600,0.608
537,NH42HP,115.025,3.0,1.14400,0.587


In [18]:
from dataclasses import dataclass
import json
import math
from openmm import unit
import numpy as np

# Replace these with real values
# volL = ...
# waters = ...
# ions_df = ...

# Helper to convert OpenMM Quantity or NumPy to float
def to_float(x):
    if isinstance(x, unit.Quantity):
        return x.value_in_unit(x.unit)
    elif isinstance(x, (np.generic, float, int)):
        return float(x)
    else:
        return x  # Pass through 'NA' or other strings

@dataclass
class SaltData:
    salt: str
    molality: float
    molarity: float
    num_particles: float
    osmotic_coefficient: float
    density: str

ion_dict = {}
data_list = []

with open("particles_results.txt", "w") as file:
    for i in range(len(ions_df)):
        row = ions_df.iloc[i]
        salt = row['Salt']
        molar_mass = row['Molar Mass (g/mol)']
        molality = row['Molality (kg/mol)']
        density = row['Density (g/mL)']
        osmotic_coefficient = row['Exp Osmotic Coefficient']

        if math.isnan(osmotic_coefficient):
            continue
        if math.isnan(density):
            density = "NA"

        def particles_no_density(
            salt,
            mm_salt,
            molality_desired,
            vol,
            num_water_molecules,
            osmotic_coefficient
        ):
            AVOGADRO = 6.02214076e23
            WATER_MOLAR_MASS = 18.01528
            KG_PER_GRAM = 1e-3

            mass_water_grams = (num_water_molecules / AVOGADRO) * WATER_MOLAR_MASS
            mass_water_kg = mass_water_grams * KG_PER_GRAM
            moles_solute = molality_desired * mass_water_kg
            grams_solute = moles_solute * mm_salt
            total_mass_solution_g = grams_solute + mass_water_grams
            molarity_final = moles_solute / vol
            density_g_per_mL = total_mass_solution_g / (vol * 1000)
            molecules = moles_solute * AVOGADRO
            moles_water = mass_water_kg * 1000 / WATER_MOLAR_MASS
            mole_fctn = moles_solute / (moles_solute + moles_water)

            result = (
                f"Salt: {salt}\n"
                f"Molality: {to_float(molality_desired):.3f} mol/kg\n"
                f"Molarity: {to_float(molarity_final):.3f} M\n"
                f"Estimated Density: {to_float(density_g_per_mL):.4f} g/mL\n"
                f"Number of particles of solute: {math.ceil(to_float(molecules))}\n"
                f"Mole Fraction: {to_float(mole_fctn):.3f}\n"
                f"Osmotic Coefficient: {to_float(osmotic_coefficient):.3f}\n"
                "--------------------------------------------\n"
            )

            return result, molecules, molarity_final, density_g_per_mL

        result, particles_number, molarity, density_est = particles_no_density(
            salt=salt,
            mm_salt=molar_mass,
            molality_desired=molality,
            vol=volL,
            num_water_molecules=math.ceil(waters),
            osmotic_coefficient=osmotic_coefficient
        )

        file.write(result)

        if salt not in ion_dict:
            ion_dict[salt] = []

        ion_dict[salt].append({
            "Molality": round(to_float(molality), 3),
            "Density": str(density),
            "Particle Number": math.ceil(to_float(particles_number)),
            "Molarity": round(to_float(molarity), 3),
            "Osmotic Coefficient": round(to_float(osmotic_coefficient), 3),
            "Result": result.strip()
        })

        data_instance = SaltData(
            salt=salt,
            molality=round(to_float(molality), 3),
            molarity=round(to_float(molarity), 3),
            num_particles=math.ceil(to_float(particles_number)),
            osmotic_coefficient=round(to_float(osmotic_coefficient), 3),
            density=str(density)
        )
        data_list.append(data_instance)

# Save the data to a Python file
with open("../salt_data.py", "w") as pyfile:
    pyfile.write("from dataclasses import dataclass\n\n")
    pyfile.write("@dataclass\n")
    pyfile.write("class SaltData:\n")
    pyfile.write("    salt: str\n")
    pyfile.write("    molality: float\n")
    pyfile.write("    molarity: float\n")
    pyfile.write("    num_particles: float\n")
    pyfile.write("    osmotic_coefficient: float\n")
    pyfile.write("    density: str\n\n")

    json_compatible_data = [
        {
            "salt": d.salt,
            "molality": round(to_float(d.molality), 3),
            "molarity": round(to_float(d.molarity), 3),
            "num_particles": d.num_particles,
            "osmotic_coefficient": round(to_float(d.osmotic_coefficient), 3),
            "density": d.density
        }
        for d in data_list
    ]
    pyfile.write(f"salt_infos = {json.dumps(json_compatible_data, indent=4)}\n")

print("Results have been written to 'particles_results.txt' and 'salt_data.py'.")


Results have been written to 'particles_results.txt' and 'salt_data.py'.


In [6]:
def get_particle_number(salt, molality):
    if salt in ion_dict:
        for entry in ion_dict[salt]:
            if entry["Molality"] == molality:
                return entry["Particle Number"]
    return None  # Return None if not found


In [13]:
# Example: Retrieve particle number for 'CsBr' at molality 0.2
salt_name = "NH42HP"
molality_value = 0.5

particle_number = get_particle_number(salt_name, molality_value)

if particle_number:
    print(f"Particle number for {salt_name} at molality {molality_value}: {particle_number:.0f}")
else:
    print(f"No data found for {salt_name} at molality {molality_value}.")


Particle number for NH42HP at molality 0.5: 32


In [25]:
salt_list=set(ions_df['Salt'])
part_4=[]
for s in salt_list:
    salt_name = s
    molality_value = 4.0

    particle_number = get_particle_number(salt_name, molality_value)

    if particle_number:
        print(f"Particle number for {salt_name} at molality {molality_value}: {particle_number:.0f}")
        part_4.append(particle_number)
    else:
        print(f"No data found for {salt_name} at molality {molality_value}.")

No data found for CsI at molality 4.0.
No data found for KNO3 at molality 4.0.
Particle number for CsBr at molality 4.0: 222
Particle number for Na2HP at molality 4.0: 232
Particle number for LiClO4 at molality 4.0: 225
No data found for CsNO3 at molality 4.0.
Particle number for NaBr at molality 4.0: 240
Particle number for NH4Cl at molality 4.0: 230
No data found for LiI at molality 4.0.
Particle number for KI at molality 4.0: 223
Particle number for NaCl at molality 4.0: 246
No data found for NaI at molality 4.0.
Particle number for KBr at molality 4.0: 231
Particle number for LiBr at molality 4.0: 240
Particle number for LiNO3 at molality 4.0: 237
Particle number for RbBr at molality 4.0: 227
Particle number for RbCl at molality 4.0: 232
Particle number for NH4NO3 at molality 4.0: 222
Particle number for LiCl at molality 4.0: 246
Particle number for NaNO3 at molality 4.0: 235
Particle number for KCl at molality 4.0: 236
Particle number for RbNO3 at molality 4.0: 223
Particle number

In [26]:
part_4=np.array(part_4)
print('%i'%part_4.mean())


231


In [27]:
from openmm.unit import Quantity, Unit, nanometer,angstrom, kilocalorie_per_mole,kilojoule_per_mole,AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
#rmin/2 to sigma

def rmin2sigma(rmin2:Quantity)-> Quantity:
    #rmin2=rmin2.in_units_of(angstrom)
    sigma=rmin2*(2**(-1/6))
    sigmanm=sigma.in_units_of(nanometer)
    print(f"Rmin to sigma= {sigmanm}")

def hrmin2sigma(rmin2:Quantity)-> Quantity:
    #rmin2=rmin2.in_units_of(angstrom)
    sigma=rmin2*(2**(-1/6))
    sigma=sigma*2
    sigmanm=sigma.in_units_of(nanometer)
    print(f"1/2Rmin to sigma= {sigmanm}")

def hrmin2sigmaA(rmin2:Quantity)-> Quantity:
    #rmin2=rmin2.in_units_of(angstrom)
    sigma=rmin2*(2**(-1/6))
    sigma=sigma*2
    print(f"1/2Rmin to sigma= {sigma}")
    return(sigma)

def emin2eps(emin:Quantity)-> Quantity:
        eminkj=emin.in_units_of(kilojoule_per_mole)
        eps=-eminkj
        print(eps)
        return(eps)

In [28]:
# na=rmin2sigma(1.369*angstrom)
# cl=rmin2sigma(2.513*angstrom)

naA=hrmin2sigma(1.369*angstrom)
clA=hrmin2sigma(2.513*angstrom)

naA=hrmin2sigmaA(1.369*angstrom)
clA=hrmin2sigmaA(2.513*angstrom)

LB_sigma=(naA+clA)/2
print(LB_sigma)

epsNa=0.0874393*(kilocalorie_per_mole)
epsNakjmol=epsNa.in_units_of(kilojoule_per_mole)
print(epsNakjmol)

epsCl=0.0355910*(kilocalorie_per_mole)
epsClkjmol=epsCl.in_units_of(kilojoule_per_mole)
print(epsClkjmol)

1/2Rmin to sigma= 0.2439280690268249 nm
1/2Rmin to sigma= 0.4477656957373345 nm
1/2Rmin to sigma= 2.439280690268249 A
1/2Rmin to sigma= 4.477656957373345 A
3.458468823820797 A
0.3658460312 kJ/mol
0.14891274399999999 kJ/mol


In [29]:
## OpenFF
csA=hrmin2sigmaA(1.976*angstrom)
brA=hrmin2sigmaA(2.608*angstrom)

LB_sigma=(csA+brA)/2
print(f'Cross interaction sigma = {LB_sigma}')

cs_eps=0.4065394*(kilocalorie_per_mole)
cs_eps_kjmol=cs_eps.in_units_of(kilojoule_per_mole)
print(f'Epsilon cation = {cs_eps_kjmol}')

br_eps=0.0586554*(kilocalorie_per_mole)
br_eps_kjmol=br_eps.in_units_of(kilojoule_per_mole)
print(f'Epsilon anion = {br_eps_kjmol}')

ceps_kjmol=np.sqrt(cs_eps_kjmol*br_eps_kjmol)
print(f'Cross interaction epsilon = {ceps_kjmol/BOLTZMANN_CONSTANT_kB/AVOGADRO_CONSTANT_NA}')

ceps=np.sqrt(cs_eps*br_eps)
print(f'Cross interaction epsilon = {ceps/BOLTZMANN_CONSTANT_kB/AVOGADRO_CONSTANT_NA}')

1/2Rmin to sigma= 3.5208317340906206 A
1/2Rmin to sigma= 4.64692771382001 A
Cross interaction sigma = 4.083879723955315 A
Epsilon cation = 1.7009608496 kJ/mol
Epsilon anion = 0.24541419360000002 kJ/mol
Cross interaction epsilon = 77.70747764074628 K
Cross interaction epsilon = 77.70747764074628 K


In [30]:
## J&CH
csA=hrmin2sigmaA(1.976*angstrom)
brA=hrmin2sigmaA(2.608*angstrom)

LB_sigma=(csA+brA)/2
print(f'Cross interaction sigma = {LB_sigma}')

cs_eps=0.4065394*(kilocalorie_per_mole)
cs_eps_kjmol=cs_eps.in_units_of(kilojoule_per_mole)
print(f'Epsilon cation = {cs_eps_kjmol}')

br_eps=0.0586554*(kilocalorie_per_mole)
br_eps_kjmol=br_eps.in_units_of(kilojoule_per_mole)
print(f'Epsilon anion = {br_eps_kjmol}')

ceps_kjmol=np.sqrt(cs_eps_kjmol*br_eps_kjmol)
print(f'Cross interaction epsilon = {ceps_kjmol}')

ceps=np.sqrt(cs_eps*br_eps)
print(f'Cross interaction epsilon = {ceps}')

1/2Rmin to sigma= 3.5208317340906206 A
1/2Rmin to sigma= 4.64692771382001 A
Cross interaction sigma = 4.083879723955315 A
Epsilon cation = 1.7009608496 kJ/mol
Epsilon anion = 0.24541419360000002 kJ/mol
Cross interaction epsilon = 0.6460959179949638 kJ/mol
Cross interaction epsilon = 0.15442063049592825 kcal/mol


In [31]:
na_e=emin2eps(-0.0469*kilocalorie_per_mole)
cl_e=emin2eps(-0.1500*kilocalorie_per_mole)

ceps=np.sqrt(na_e*cl_e)
print(ceps)

0.1962296 kJ/mol
0.6276 kJ/mol
0.3509326102829431 kJ/mol


In [32]:
minNa=-0.0469*kilocalorie_per_mole
eminCl=-0.1500*kilocalorie_per_mole
combemin=np.sqrt(eminNa*eminCl)
print(combemin.in_units_of(kilojoule_per_mole))

NameError: name 'eminNa' is not defined

In [ ]:
epsna=0.0874393*kilocalorie_per_mole
epsna=epsna.in_units_of(kilojoule_per_mole)

epscl=0.0355910*kilocalorie_per_mole
epscl=epscl.in_units_of(kilojoule_per_mole)

print(epsna)
print(epscl)

0.3658460312 kJ/mol
0.14891274399999999 kJ/mol
